# Error Handling
> I knew if I stayed around long enough, something like this would happen.
>         —George Bernard Shaw on dying

Ordinary errors are handled using the Result type. Results typically represent prob‐ lems caused by things outside the program, like erroneous input, a network outage, or a permissions problem. That such situations occur is not up to us; even a bug-free program will encounter them from time to time. Most of this chapter is dedicated to that kind of error. We’ll cover panic first, though, because it’s the simpler of the two.


Panic is for the other kind of error, the kind that should never happen.

## Panic
A program panics when it encounters something so messed up that there must be a bug in the program itself. Something like:
* Out-of-bounds array access
* Integer division by zero
* Calling `.expect()` on a Result that happens to be `Err`
* Assertion failure

(There’s also the macro `panic!()`, for cases where your own code discovers that it has gone wrong, and you therefore need to trigger a panic directly. `panic!()` accepts optional `println!()`-style arguments, for building an error message.)

But we all make mistakes. When these errors that shouldn’t happen do happen—what then? Remarkably, Rust gives you a choice. Rust can either unwind the stack when a panic happens or abort the process. Unwinding is the default.

### Unwinding


In Rust, it triggers a panic, which typically proceeds as follows:
* An error message is printed to the terminal
* The stack is unwound. This is a lot like C++ exception handling.
Any temporary values, local variables, or arguments that the current function was using are dropped, in the reverse of the order they were created. Dropping a value simply means cleaning up after it: any Strings or Vecs the program was using are freed, any open Files are closed, and so on. User-defined drop methods are called too. Once the current function call is cleaned up, we move on to its caller, dropping its variables and arguments the same way. Then we move to that function’s caller, and so on up the stack.
* Finally, the thread exits. If the panicking thread was the main thread, then the whole process exits (with a nonzero exit code).

Perhaps panic is a misleading name for this orderly process. A panic is not a crash. It’s not undefined behavior. It’s more like a `RuntimeException` in Java or a `std::logic_error` in C++. The behavior is well-defined; it just shouldn’t be happening.

Panic is safe. It doesn’t violate any of Rust’s safety rules; even if you manage to panic in the middle of a standard library method, it will never leave a dangling pointer or a half-initialized value in memory. The idea is that Rust catches the invalid array access, or whatever it is, before anything bad happens. It would be unsafe to proceed, so Rust unwinds the stack. But the rest of the process can continue running.

Panic is per thread. One thread can be panicking while other threads are going on about their normal business.

There is also a way to catch stack unwinding, allowing the thread to survive and continue running. The standard library function `std::panic::catch_unwind()` does this. 

### Aborting
Stack unwinding is the default panic behavior, but there are two circumstances in which Rust does not try to unwind the stack.
If a `.drop()` method triggers a second panic while Rust is still trying to clean up after the first, this is considered fatal. Rust stops unwinding and aborts the whole process.
Also, Rust’s panic behavior is customizable. If you compile with `-C panic=abort`, the first panic in your program immediately aborts the process. (With this option, Rust does not need to know how to unwind the stack, so this can reduce the size of your compiled code.)

## Result
Rust doesn’t have exceptions. Instead, functions that can fail have a return type that says so:
```Rust
fn get_weather(location: LatLng) -> Result<WeatherReport, io::Error>
```
The Result type indicates possible failure. When we call the `get_weather()` function, it will return either a success result `Ok(weather)`, where weather is a new `WeatherReport` value, or an error result `Err(error_value)`, where `error_value` is an `io::Error` explaining what went wrong.
Rust requires us to write some kind of error handling whenever we call this function. We can’t get at the `WeatherReport` without doing something to the Result, and you’ll get a compiler warning if a Result value isn’t used.

### Catching Errors
The most thorough way of dealing with a `Result`:
```Rust
match get_weather(hometown) { 
    Ok(report) => {
        display_weather(hometown, &report);
    }
    Err(err) => {
        println!("error querying the weather: {}", err);
        schedule_weather_retry();
    } 
}
```
This is Rust’s equivalent of `try/catch` in other languages. It’s what you use when you want to handle errors head-on, not pass them on to your caller.

match is a bit verbose, so `Result<T, E>` offers a variety of methods that are useful in particular common cases. Each of these methods has a match expression in its implementation.

### Result Type Aliases
Sometimes you’ll see Rust documentation that seems to omit the error type of a Result:
```Rust
fn remove_file(path: &Path) -> Result<()>
```
This means that a Result type alias is being used.

A type alias is a kind of shorthand for type names. Modules often define a Result type alias to avoid having to repeat an error type that’s used consistently by almost every function in the module. For example, the standard library’s `std::io` module includes this line of code:
```Rust
pub type Result<T> = result::Result<T, Error>;
```
This defines a public type `std::io::Result<T>`. It’s an alias for `Result<T, E>`, but hardcodes `std::io::Error` as the error type. In practical terms, this means that if you write use `std::io`;, then Rust will understand `io::Result<String>` as shorthand for `Result<String, io::Error>`.
When something like `Result<()>` appears in the online documentation, you can click on the identifier `Result` to see which type alias is being used and learn the error type. In practice, it’s usually obvious from context.

### Printing Errors
Sometimes the only way to handle an error is by dumping it to the terminal and moving on. We already showed one way to do this:
```Rust
println!("error querying the weather: {}", err);
```
The standard library defines several error types with boring names: `std::io::Error`, `std::fmt::Error`, `std::str::Utf8Error`, and so on. All of them implement a common interface, the `std::error::Error` trait.

Printing an error value does not also print out its source. If you want to be sure to print all the available information, use this function:
```Rust
use std::error::Error;
use std::io::{Write, stderr};
/// Dump an error message to `stderr`.
///
/// If another error happens while building the error message or 
/// writing to `stderr`, it is ignored.
fn print_error(mut err: &dyn Error) {
    let _ = writeln!(stderr(), "error: {}", err); 
    while let Some(source) = err.source() {
        let _ = writeln!(stderr(), "caused by: {}", source);
        err = source;
    }
}
```

The `writeln!` macro works like `println!`, except that it writes the data to a stream of your choice. Here, we write the error messages to the standard error stream, `std::io::stderr`. We could use the `eprintln!` macro to do the same thing, but `eprintln!` panics if an error occurs. In `print_error`, we want to ignore errors that arise while writing the message.

### Propagating Errors
In most places where we try something that could fail, we don’t want to catch and handle the error immediately. It is simply too much code to use a 10-line match statement every place where something could go wrong.

Instead, if an error occurs, we usually want to let our caller deal with it. We want errors to _propagate_ up the call stack.

Rust has a `?` operator that does this. You can add a `?` to any expression that produces a `Result`, such as the result of a function call:
```Rust
let weather = get_weather(hometown)?;
```
The behavior of `?` depends on whether this function returns a success result or an
error result:
* On success, it unwraps the Result to get the success value inside. The type of weather here is not `Result<WeatherReport, io::Error>` but simply WeatherReport.
* On error, it immediately returns from the enclosing function, passing the error result up the call chain. To ensure that this works, `?` can only be used on a Result in functions that have a `Result` return type.

`?` also works similarly with the `Option` type. In a function that returns Option, you can use `?` to unwrap a value and return early in the case of None:


### Working with Multiple Error Types

In [3]:
use std::io::{self, BufRead};
/// Read integers from a text file.
/// The file should have one number on each line.
fn read_numbers(file: &mut dyn BufRead) -> Result<Vec<i64>, io::Error> {
    let mut numbers = vec![];
    for line_result in file.lines() {
        let line = line_result?; 
        numbers.push(line.parse()?);
    }
    Ok(numbers)
}

Error: `?` couldn't convert the error to `std::io::Error`

The problem here is that reading a line from a file and parsing an integer produce two different potential error types. The type of `line_result` is `Result<String, std::io::Error>`. The type of `line.parse()` is `Result<i64, std::num::Parse IntError>`. The return type of our `read_numbers()` function only accommodates `io::Errors`. Rust tries to cope with the ParseIntError by converting it to a `io::Error`, but there’s no such conversion, so we get a type error.

There are several ways of dealing with this:
* The `thiserror crate`, which is designed to help you define good error types with just a few lines of code.
* A simpler approach is to use what’s built into Rust. All of the standard library error types can be converted to the type `Box<dyn std::error::Error + Send + Sync + 'static>`. This is a bit of a mouthful, but `dyn std::error::Error` represents “any error,” and `Send + Sync + 'static` makes it safe to pass between threads, which you’ll often want. For convenience, you can define type aliases:

In [4]:
use std::io::{self, BufRead};

type GenericError = Box<dyn std::error::Error + Send + Sync + 'static>; 
type GenericResult<T> = Result<T, GenericError>;
/// Read integers from a text file.
/// The file should have one number on each line.
fn read_numbers(file: &mut dyn BufRead) -> GenericResult<Vec<i64>> {
    let mut numbers = vec![];
    for line_result in file.lines() {
        let line = line_result?; 
        numbers.push(line.parse()?);
    }
    Ok(numbers)
}

Incidentally, the `?` operator does this automatic conversion using a standard method that you can use yourself. To convert any error to the `GenericError` type, call `GenericError::from()`:
```Rust
let io_error = io::Error::new( // make our own io::Error io::ErrorKind::Other, "timed out");
return Err(GenericError::from(io_error)); // manually convert to GenericError
```
The downside of the `GenericError` approach is that the return type no longer communicates precisely what kinds of errors the caller can expect. The caller must be ready for anything.

If you’re calling a function that returns a GenericResult and you want to handle one particular kind of error but let all others propagate out, use the generic method `error.downcast_ref::<ErrorType>()`. It borrows a reference to the error, if it happens to be the particular type of error you’re looking for:
```Rust
loop {
    match compile_project() {
        Ok(()) => return Ok(()), 
        Err(err) => {
            if let Some(mse) = err.downcast_ref::<MissingSemicolonError>() { 
                insert_semicolon_in_source_code(mse.file(), mse.line())?; continue; // try again!
            }
            return Err(err); 
        }
    } 
}
```
Many languages have built-in syntax to do this, but it turns out to be rarely needed. Rust has a method for it instead.

### Dealing with Errors That “Can’t Happen”
Sometimes we just know that an error can’t happen. For example, suppose we’re writing code to parse a configuration file, and at one point we find that the next thing in the file is a string of digits:
```Rust
if next_char.is_digit(10) {
    let start = current_index;
    current_index = skip_digits(&line, current_index); 
    let digits = &line[start..current_index];
    ...
```
We want to convert this string of digits to an actual number. There’s a standard method that does this:
```Rust
let num = digits.parse::<u64>();
```
Now the problem: the `str.parse::<u64>()` method doesn’t return a `u64`. It returns a
Result. It can fail, because some strings aren’t numeric: 
```Rust
"bleen".parse::<u64>() // ParseIntError: invalid digit
```
But we happen to know that in this case, digits consists entirely of digits. What should we do?

If the code we’re writing already returns a GenericResult, we can tack on a `?` and forget about it. Otherwise, we face the irritating prospect of having to write errorhandling code for an error that can’t happen. The best choice then would be to use `.unwrap()`, a `Result` method that panics if the result is an `Err`, but simply returns the success value of an `Ok`:
```Rust
let num = digits.parse::<u64>().unwrap();
```
This is just like `?` except that if we’re wrong about this error, if it can happen, then in
that case we would panic.

In [10]:
"99999999999999999999".parse::<u64>().unwrap(); // overflow error

thread '<unnamed>' panicked at 'called `Result::unwrap()` on an `Err` value: ParseIntError { kind: PosOverflow }', src/lib.rs:20:39
stack backtrace:
   0: _rust_begin_unwind
   1: core::panicking::panic_fmt
   2: core::result::unwrap_failed
   3: <unknown>
   4: evcxr::runtime::Runtime::run_loop
   5: evcxr::runtime::runtime_hook
   6: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


In [15]:
"99999999999999999999".parse::<u64>()

Err(ParseIntError { kind: PosOverflow })

In [17]:
"999999999".parse::<u64>() // overflow error

Ok(999999999)

Using `.unwrap()` in this particular case would therefore be a bug. Bogus input shouldn’t cause a panic.

### Ignoring Errors
Occasionally we just want to ignore an error altogether. For example, in our `print_error()` function, we had to handle the unlikely situation where printing the error triggers another error. This could happen, for example, if `stderr` is piped to another process, and that process is killed. The original error we were trying to report is probably more important to propagate, so we just want to ignore the troubles with `stderr`, but the Rust compiler warns about unused Result values:
```Rust
writeln!(stderr(), "error: {}", err); // warning: unused result
```
The idiom let _ = ... is used to silence this warning:
```Rust
let _ = writeln!(stderr(), "error: {}", err); // ok, ignore result
```

### Handling Errors in main()
In most places where a Result is produced, letting the error bubble up to the caller is the right behavior. This is why `?` is a single character in Rust. As we’ve seen, in some programs it’s used on many lines of code in a row.

But if you propagate an error long enough, eventually it reaches `main()`, and something has to be done with it. Normally, `main()` can’t use `?` because its return type is not `Result`:
```Rust
fn main() {
    calculate_tides()?; // error: can't pass the buck any further
}
```
The simplest way to handle errors in main() is to use `.expect()`:
```Rust
fn main() {
    calculate_tides().expect("error"); // the buck stops here
}
```
If `calculate_tides()` returns an error result, the `.expect()` method panics. Panicking in the main thread prints an error message and then exits with a nonzero exit code, which is roughly the desired behavior. We use this all the time for tiny programs. It’s a start.

The error message is a little intimidating, though:
```bash
$ tidecalc --planet mercury
thread 'main' panicked at 'error: "moon not found"', src/main.rs:2:23
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace
```
The error message is lost in the noise. Also, `RUST_BACKTRACE=1` is bad advice in this particular case.
However, you can also change the type signature of `main()` to return a `Result` type, so you can use `?`:
```Rust
fn main() -> Result<(), TideCalcError> { 
    let tides = calculate_tides()?; 
    print_tides(tides);
    Ok(())
}
```
This works for any error type that can be printed with the `{:?}` formatter, which all standard error types, like `std::io::Error`, can be. This technique is easy to use and gives a somewhat nicer error message, but it’s not ideal:
```bash
$ tidecalc --planet mercury
Error: TideCalcError { error_type: NoMoon, message: "moon not found" }
```
If you have more complex error types or want to include more details in your message, it pays to print the error message yourself:
```Rust
fn main() {
    if let Err(err) = calculate_tides() {
        print_error(&err);
        std::process::exit(1);
    }
}
```

### Declaring a Custom Error Type
Suppose you are writing a new JSON parser, and you want it to have its own error type.
Approximately the minimum code you would write is:
```Rust
// json/src/error.rs
#[derive(Debug, Clone)]
pub struct JsonError { 
    pub message: String, 
    pub line: usize, 
    pub column: usize,
}
```
This struct will be called `json::error::JsonError`, and when you want to raise an error of this type, you can write:
```Rust
return Err(JsonError {
    message: "expected ']' at end of array".to_string(), 
    line: current_line,
    column: current_column
});
```
This will work fine. However, if you want your error type to work like the standard error types, as your library’s users will expect, then you have a bit more work to do:
```Rust
use std::fmt;
// Errors should be printable.
impl fmt::Display for JsonError {
    fn fmt(&self, f: &mut fmt::Formatter) -> Result<(), fmt::Error> {
        write!(f, "{} ({}:{})", self.message, self.line, self.column)
    }
}
// Errors should implement the std::error::Error trait,
// but the default definitions for the Error methods are fine. 
impl std::error::Error for JsonError { }
```
As with many aspects of the Rust language, crates exist to make error handling much easier and more concise. There is quite a variety, but one of the most used is `thiserror`, which does all of the previous work for you, allowing you to write errors like this:
```Rust
use thiserror::Error;

#[derive(Error, Debug)] 
#[error("{message:} ({line:}, {column})")]
pub struct JsonError { 
    message: String,
    line: usize,
    column: usize, 
}
```
The `#[derive(Error)]` directive tells `thiserror` to generate the code shown earlier, which can save a lot of time and effort.